In [57]:
# In this workshop we perform document clustering using sklearn

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# We are using the subnews dataset that we used last week. 
# The "Class" labels here are only used for sanity check of the clusters found later.
# Remember, in actual use of document clustering, the documents DON'T come with labeled classes.
# It's unsupervised learning.

import pandas as pd
news=pd.read_table('r8-train-all-terms.txt',header=None,names = ["Class", "Text"])
subnews=news[(news.Class=="trade")| (news.Class=='crude')|(news.Class=='money-fx') ]
subnews.head()

,Class,Text
15,trade,brazil anti inflation plan limps to anniversar...
43,crude,diamond shamrock dia cuts crude prices diamond...
55,crude,opec may have to meet to firm prices analysts ...
76,crude,texaco canada cuts crude prices canadian cts b...
77,crude,texaco canada txc lowers crude postings texaco...


In [58]:
# Let's use the similar preprocessing we used last week.
# The output of each document is a list of tokens.

import nltk
from nltk.corpus import stopwords
mystopwords=stopwords.words("English") + ['one', 'become', 'get', 'make', 'take']
WNlemma = nltk.WordNetLemmatizer()

def pre_process(text):
    tokens = nltk.word_tokenize(text)
    tokens=[ WNlemma.lemmatize(t.lower()) for t in tokens]
    tokens=[ t for t in tokens if t not in mystopwords]
    tokens = [ t for t in tokens if len(t) >= 3 ]
    text_after_process=" ".join(tokens)
    return(text_after_process)

# Apply preprocessing to every document in the training set.
text = subnews['Text']
toks = text.apply(pre_process)

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline


In [60]:
# Create tfidf matrix
vectorizer = TfidfVectorizer(max_df=0.7, max_features=2500,
                             min_df=3, stop_words=mystopwords,
                             use_idf=True)
X = vectorizer.fit_transform(toks)
X.shape

(710, 2500)

In [61]:
# Use SVD to reduce dimensions
svd = TruncatedSVD(300)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X_lsa = lsa.fit_transform(X)

In [62]:
# Check how much variance is explained
explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))


Explained variance of the SVD step: 85%


In [63]:
# Now the actual clustering
from sklearn.cluster import KMeans

km3 = KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=1)
%time km3.fit(X_lsa)


CPU times: user 29.6 ms, sys: 908 µs, total: 30.5 ms
Wall time: 16.9 ms


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=3, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [64]:
# How do we know the clustering result is good or not?
# If we have labels available, we can use this to derive how coherent the clusters are.
# Homogeneity: each cluster contains only members of a single class

from sklearn import metrics

labels = subnews['Class']
print("Homogeneity for 3 clusters: %0.3f" % metrics.homogeneity_score(labels, km3.labels_))

Homogeneity for 3 clusters: 0.602


In [65]:
# Let's try some other K values to compare their metrics
km2 = KMeans(n_clusters=2, init='k-means++', max_iter=100, n_init=1)
%time km2.fit(X_lsa)

km4 = KMeans(n_clusters=4, init='k-means++', max_iter=100, n_init=1)
%time km4.fit(X_lsa)

km5 = KMeans(n_clusters=5, init='k-means++', max_iter=100, n_init=1)
%time km5.fit(X_lsa)

CPU times: user 16.6 ms, sys: 1.29 ms, total: 17.9 ms
Wall time: 9.1 ms
CPU times: user 54.6 ms, sys: 488 µs, total: 55.1 ms
Wall time: 27.8 ms
CPU times: user 33.4 ms, sys: 121 µs, total: 33.6 ms
Wall time: 16.7 ms


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=5, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [66]:
# What are their homogeneity scores?
print("Homogeneity for 2 clusters: %0.3f" % metrics.homogeneity_score(labels, km2.labels_))
print("Homogeneity for 4 clusters: %0.3f" % metrics.homogeneity_score(labels, km4.labels_))
print("Homogeneity for 5 clusters: %0.3f" % metrics.homogeneity_score(labels, km5.labels_))


Homogeneity for 2 clusters: 0.543
Homogeneity for 4 clusters: 0.832
Homogeneity for 5 clusters: 0.615


In [67]:
# Completeness: all members of a given class are assigned to the same cluster

print("Completeness for 2 clusters: %0.3f" % metrics.completeness_score(labels, km2.labels_))
print("Completeness for 3 clusters: %0.3f" % metrics.completeness_score(labels, km3.labels_))
print("Completeness for 4 clusters: %0.3f" % metrics.completeness_score(labels, km4.labels_))
print("Completeness for 5 clusters: %0.3f" % metrics.completeness_score(labels, km5.labels_))

Completeness for 2 clusters: 0.922
Completeness for 3 clusters: 0.751
Completeness for 4 clusters: 0.715
Completeness for 5 clusters: 0.488


In [68]:
# Silhouette: more similar within clusters, more distant between clusters
# The higher the better
print("Silhouette Coefficient for 2 clusters: %0.3f"
      % metrics.silhouette_score(X_lsa, km2.labels_))
print("Silhouette Coefficient for 3 clusters: %0.3f"
      % metrics.silhouette_score(X_lsa, km3.labels_))
print("Silhouette Coefficient for 4 clusters: %0.3f"
      % metrics.silhouette_score(X_lsa, km4.labels_))
print("Silhouette Coefficient for 5 clusters: %0.3f"
      % metrics.silhouette_score(X_lsa, km5.labels_))

Silhouette Coefficient for 2 clusters: 0.031
Silhouette Coefficient for 3 clusters: 0.050
Silhouette Coefficient for 4 clusters: 0.055
Silhouette Coefficient for 5 clusters: 0.054


In [69]:
# We still need to see the more representative words for each cluster to understand them.

def print_terms(cm, num):
    original_space_centroids = svd.inverse_transform(cm.cluster_centers_)
    order_centroids = original_space_centroids.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(num):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()


In [70]:
# Print the terms for the 2-cluster model
print_terms(km2, 2)

Cluster 0: oil crude price opec barrel dlrs mln bpd company ecuador
Cluster 1: trade billion bank stg japan mln market exchange rate money


In [71]:
print_terms(km3, 3)

Cluster 0: stg mln money bank england market revised assistance shortage forecast
Cluster 1: trade billion japan bank exchange dollar currency rate japanese would
Cluster 2: oil crude price opec barrel dlrs mln bpd company ecuador


In [72]:
print_terms(km4, 4)

Cluster 0: bank currency exchange rate dollar fed say baker paris treasury
Cluster 1: stg mln money bank england market revised assistance shortage forecast
Cluster 2: oil crude price opec barrel dlrs mln bpd company ecuador
Cluster 3: trade japan billion japanese surplus export year import deficit would


In [73]:
print_terms(km5, 5)

Cluster 0: crude bbl price dlrs raise wti posting posted effective raised
Cluster 1: stg mln money bank england market revised assistance shortage forecast
Cluster 2: oil opec barrel mln price bpd crude ecuador saudi energy
Cluster 3: trade japan bank exchange rate currency dollar japanese market baker
Cluster 4: billion trade dlrs fed deficit surplus january bill export february


In [76]:
# Let's map the cluster label to the categories to see where is the confusion

dict = {2: 'crude', 0: 'money-fx', 1: 'trade'}
cluster_labels = [ dict[c] for c in km3.labels_]

In [77]:
import numpy as np
print(metrics.confusion_matrix(cluster_labels, labels))
print(np.mean(cluster_labels == labels) )
print(metrics.classification_report(cluster_labels, labels))

[[245   0   0]
 [  1  52   0]
 [  7 154 251]]
0.771830985915
             precision    recall  f1-score   support

      crude       0.97      1.00      0.98       245
   money-fx       0.25      0.98      0.40        53
      trade       1.00      0.61      0.76       412

avg / total       0.93      0.77      0.81       710

